In [1]:
config_path = "test\\Config.xlsx"
OUTPUT_PATH = "test\\Processed"
data_path = "test\\data\\2020 Novartis SOW_Capmatinib US _V19.xlsx"
# data_path = "Check\\data"
log_path = "test\\Logs"

In [2]:
from ExcelParser.MakeDF import CleanDFByConfig
from io import BytesIO

In [3]:
df = None

    
cdf = CleanDFByConfig(config_path)

In [4]:
with open(data_path, "rb") as f:
    in_mem_file = BytesIO(f.read())
    df = cdf.get_df(data_path, bytesIO=in_mem_file, sheet_name='sow desc', excel_table=True)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\ops.py:1649: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


{'project name|project desc|project assump': {'coordinate': (22, 3), 'cell_value': 'Project Name'}}
Directly read dataframe
Extract Direct Table
Header is : 22 and sheet name is 1. SOW Description


In [5]:
df

,Deliverable Type,Quantity,Type Of Work,End Date,Deliverable,Currency,Project Description,Hours,Fee,Project Name,...,Novartis Division,Lead Agency,Lead Agency Currency,Discipline,Other Discipline,Brand,Other Brand,Country,Modification Date,Project Version
1,Research - Creative/Strategy/Messaging,1.0,Origination,2020-12-31 00:00:00,Research,USA (USD),Attendance at key market research initiatives ...,220,57450,Market Research Attendance,...,Oncology,DDB Health (US),$ (USD),Professional,,Capmatinib,⌂,USA (USD),2019-01-03 00:00:00,V19
2,NO PROJECT TYPE,1.0,Origination,2020-12-31 00:00:00,Other,USA (USD),"Includes travel to market research, congresses...",0,0,Travel,...,Oncology,DDB Health (US),$ (USD),Professional,,Capmatinib,⌂,USA (USD),2019-01-03 00:00:00,V19
3,Brand Planning,1.0,Origination,2020-12-31 00:00:00,Strategy,USA (USD),Support for overall brand plan development alo...,125,35065,2021 Brand Planning,...,Oncology,DDB Health (US),$ (USD),Professional,,Capmatinib,⌂,USA (USD),2019-01-03 00:00:00,V19
4,Workshop,1.0,Origination,2020-12-31 00:00:00,Event_Training_Meeting,USA (USD),Time allocation for workshop attendance and pa...,102,24977,Pre-launch Objection/Concern Handling Workshop,...,Oncology,DDB Health (US),$ (USD),Professional,,Capmatinib,⌂,USA (USD),2019-01-03 00:00:00,V19
5,Campaign concept development,1.0,Origination,2020-12-31 00:00:00,Creative,USA (USD),Finalization of professional promotional campa...,265,49450,Branded Campaign Finalization,...,Oncology,DDB Health (US),$ (USD),Professional,,Capmatinib,⌂,USA (USD),2019-01-03 00:00:00,V19
6,Visual Aid - Print,1.0,Origination,2020-12-31 00:00:00,Print,USA (USD),Finalization of visual aid in preparation for ...,750,128415,Core Vis Aid Finalization,...,Oncology,DDB Health (US),$ (USD),Professional,,Capmatinib,⌂,USA (USD),2019-01-03 00:00:00,V19
7,Veeva E-Mail / eDM for Veeva Platform,1.0,Origination,2020-12-31 00:00:00,CRM,USA (USD),Day 1 email to be sent upon approval by sales ...,177,26496,Now Approved iRep Email,...,Oncology,DDB Health (US),$ (USD),Professional,,Capmatinib,⌂,USA (USD),2019-01-03 00:00:00,V19
8,Flashcard,1.0,Origination,2020-12-31 00:00:00,Print,USA (USD),Day 1 flashcard to be utilized by sales force ...,184,28978,Now Approved Flashcard,...,Oncology,DDB Health (US),$ (USD),Professional,,Capmatinib,⌂,USA (USD),2019-01-03 00:00:00,V19
9,Brochure,1.0,Origination,2020-12-31 00:00:00,Print,USA (USD),Completion of AE Management & Dosing brochure ...,287,43168,Dosing Guide & AE Management Brochure 2020 Com...,...,Oncology,DDB Health (US),$ (USD),Professional,,Capmatinib,⌂,USA (USD),2019-01-03 00:00:00,V19
10,Promo Materials - Digital,1.0,Adaptation,2020-12-31 00:00:00,Web_Digital,USA (USD),Reformatting of Vis Aid as a clickable PDF to ...,197,27626,Clickable PDF of Vis Aid,...,Oncology,DDB Health (US),$ (USD),Professional,,Capmatinib,⌂,USA (USD),2019-01-03 00:00:00,V19


In [6]:
df.iloc[0]

Deliverable Type                   Research - Creative/Strategy/Messaging
Quantity                                                                1
Type Of Work                                                  Origination
End Date                                              2020-12-31 00:00:00
Deliverable                                                      Research
Currency                                                        USA (USD)
Project Description     Attendance at key market research initiatives ...
Hours                                                                 220
Fee                                                                 57450
Project Name                                   Market Research Attendance
Total                                                               62450
Project Assumption      Assumes:\n* Attendance live for up to 2 people...
OOP                                                                  5000
Start Date                            

In [7]:
p = re.compile('created.*edited date')
print(p)

NameError: name 're' is not defined

In [ ]:
p.search("SOW Created/Edited Date: [MM/DD/YY]".lower())

In [8]:
callable(min)

True

In [9]:
class check:
    hello = "Am I visible"

In [11]:
dir(check)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'hello']